# Blazing Hawks
Clifford Jones\
Dongyu Liu\
Yuan Chen


**A.I. Disclaimer: Work for this assignment was completed with the aid of artificial intelligence tools.**



## Bisection Method vs Newton Method
First come with the example of one variable function


In [2]:
import numpy as np
import matplotlib.pyplot as plt


In [15]:


def f(x):
    # Cube root function, works for negative x too.
    return np.cbrt(x)


def bisection_method(a, b, tol=1e-8, max_iter=100):
    if f(a)*f(b) >= 0:
        print("Bisection method fails: f(a) and f(b) must have opposite signs.")
        return None
    print("\nBisection Method Iterations:")
    for i in range(max_iter):
        c = (a + b) / 2.0
        fc = f(c)
        if abs(fc) < tol or (b - a)/2 < tol:
            print("converged after", i, "iterations")
            return c
        if f(a)*fc < 0:
            b = c
        else:
            a = c
    return (a + b) / 2.0


# Bisection method: choose an interval that brackets the root.
a, b = -1, 1  # f(-1) = -1, f(1) = 1 so the sign change condition holds.
bisection_root = bisection_method(a, b)
print("\nBisection method result:", bisection_root)



Bisection Method Iterations:
converged after 0 iterations

Bisection method result: 0.0


In [ ]:

def fprime(x):
    # Derivative: 1/(3*(|x|^(2/3))).
    # Avoid division by zero.
    if x == 0:
        raise ZeroDivisionError("Derivative undefined at x=0")
    return 1 / (3 * np.cbrt(x**2))

def newton_method(x0, tol=1e-8, max_iter=20):
    x = x0
    print("Newton's Method Iterations:")
    for i in range(max_iter):
        try:
            fp = fprime(x)
        except ZeroDivisionError:
            print(f"Iteration {i}: Derivative zero at x={x}.")
            return None
        x_new = x - f(x) / fp
        print(f"Iteration {i}: x = {x_new}")
        if abs(x_new - x) < tol:
            print(f"Converged after {i} iterations.")
            return x_new
        x = x_new
    return x

# For f(x) = cbrt(x), the only root is at x = 0.
# Newton's method starting from x0 != 0 will not converge.
x0 = 1.0
newton_root = newton_method(x0)
print("\nNewton's method result:", newton_root)

## For mult variables. 
In these two methods, we can only use Newton method with Hessian Matrix for multiple variables questions.\
Slove the equation parameters with least square method.

In [22]:
np.random.seed(42)
m, n = 100, 3
X = np.random.rand(m, n)
true_beta = np.array([3,5,6])
y = X @ true_beta + np.random.randn(m)*0.1



def Newton_method(tol=1e-8, max_iter=20):
    beta = np.zeros(n)
    for i in range(max_iter):
        print(f"{i} iteration: beta = {beta}")
        gradient = X.T @ (X @ beta - y)
        hessian = X.T @ X
        beta -= np.linalg.solve(hessian, gradient)
        if np.linalg.norm(gradient) < tol:
            return beta

    print("Newton's method did not converge.")
    return beta

beta = Newton_method()
print("Estimated beta:", beta)
print("True beta:", true_beta)
    

    


0 iteration: beta = [0. 0. 0.]
1 iteration: beta = [3.01124533 4.97041046 6.03920518]
Estimated beta: [3.01124533 4.97041046 6.03920518]
True beta: [3 5 6]


In [23]:
np.random.seed(42)
m, n = 100, 2 # 100 samples, 2 features
X = np.random.rand(m, n)
true_theta = np.array([3, 5]) # true parameters
y_prob = 1 / (1 + np.exp(-X @ true_theta))  # logistic function
y = (np.random.rand(m) < y_prob).astype(int)  

In [24]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def logistic_regression_newton(X, y, tol=1e-6, max_iter=10):
    m, n = X.shape
    theta = np.zeros(n)  # initialize guess at 0
    
    for i in range(max_iter):
        h = sigmoid(X @ theta)
        
        # get gradient and Hessian matrix
        gradient = X.T @ (y - h)
        
        R = np.diag(h * (1 - h))  # m x m diagonal matrix
        H = X.T @ R @ X  # Hessian matrix

        # update theta
        delta_theta = np.linalg.solve(H, gradient)
        theta += delta_theta

        # check convergence
        if np.linalg.norm(delta_theta) < tol:
            print(f"Converged in {i+1} iterations.")
            return theta
    
    print("Newton's method did not fully converge.")
    return theta

In [25]:
theta_estimated = logistic_regression_newton(X, y)
print("Estimated theta:", theta_estimated)

Converged in 9 iterations.
Estimated theta: [2.89835968 8.0589256 ]


## QR decomposition

In [16]:
import numpy as np

### Gram-Schmidt

In [17]:
def gram_schmidt(A):
    """
    Compute the QR factorization of A using the classical Gram–Schmidt process.
    Returns:
       Q: m x n matrix with orthonormal columns
       R: n x n upper triangular matrix, with A = Q @ R
    """
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))
    for j in range(n):
        v = A[:, j].copy()
        for i in range(j):
            R[i, j] = np.dot(Q[:, i], A[:, j])
            v = v - R[i, j] * Q[:, i]
        R[j, j] = np.linalg.norm(v)
        if np.isclose(R[j, j], 0.0):
            raise ValueError("Matrix A has linearly dependent columns.")
        Q[:, j] = v / R[j, j]
    return Q, R

### Householder

In [18]:
def householder_qr(A):
    """
    Compute the QR factorization of A using Householder reflections.
    Returns:
       Q: m x m orthogonal matrix
       R: m x n upper triangular matrix, with A = Q @ R
    """
    m, n = A.shape
    R = A.copy().astype(float)
    Q = np.eye(m)

    for k in range(min(m, n)):
        # Form the vector to reflect
        x = R[k:, k]
        norm_x = np.linalg.norm(x)
        if np.isclose(norm_x, 0):
            continue
        # Choose sign to avoid cancellation
        sign = -np.sign(x[0]) if x[0] != 0 else -1
        u1 = x[0] - sign * norm_x
        v = x.copy()
        v[0] = u1
        v = v / np.linalg.norm(v)

        # Build the Householder matrix H_k for the submatrix
        Hk = np.eye(m)
        Hk[k:, k:] -= 2.0 * np.outer(v, v)

        R = Hk @ R
        Q = Q @ Hk

    return Q, R

### Given Rotation

In [19]:

def givens_qr(A):
    """
    Compute the QR factorization of A using Givens rotations.
    Returns:
       Q: m x m orthogonal matrix
       R: m x n upper triangular matrix, with A = Q @ R
    """
    m, n = A.shape
    R = A.copy().astype(float)
    Q = np.eye(m)

    # Process each column
    for j in range(n):
        # Zero out the entries below the diagonal in column j
        for i in range(j+1, m):
            a = R[j, j]
            b = R[i, j]
            r = np.hypot(a, b)
            if np.isclose(r, 0):
                continue
            c = a / r
            s = b / r
            # Apply rotation to R: update rows j and i for columns j:n
            for k in range(j, n):
                temp = c * R[j, k] + s * R[i, k]
                R[i, k] = -s * R[j, k] + c * R[i, k]
                R[j, k] = temp
            # Apply rotation to Q: update columns j and i of Q
            temp_j = Q[:, j].copy()
            temp_i = Q[:, i].copy()
            Q[:, j] = c * temp_j + s * temp_i
            Q[:, i] = -s * temp_j + c * temp_i
    return Q, R


In [20]:

# Main script to compute and compare QR factorizations
if __name__ == "__main__":
    np.random.seed(0)
    A = np.random.randn(4, 3)

    print("Matrix A:")
    print(A)

    # 1. Gram-Schmidt QR factorization (reduced Q: m x n)
    Q_gs, R_gs = gram_schmidt(A)
    print("\n--- Gram-Schmidt QR Factorization ---")
    print("Q (Gram-Schmidt):")
    print(Q_gs)
    print("R (Gram-Schmidt):")
    print(R_gs)
    print("Reconstruction (Q_gs @ R_gs):")
    print(Q_gs @ R_gs)

    # 2. Householder QR factorization (full Q: m x m)
    Q_h_full, R_h = householder_qr(A)
    # Extract the first n columns for comparison
    Q_h = Q_h_full[:, :A.shape[1]]
    print("\n--- Householder QR Factorization ---")
    print("Q (Householder, reduced to first n columns):")
    print(Q_h)
    print("R (Householder):")
    print(R_h)
    print("Reconstruction (Q_h_full @ R_h):")
    print(Q_h_full @ R_h)

    # 3. Givens QR factorization (full Q: m x m)
    Q_giv_full, R_giv = givens_qr(A)
    # Extract the first n columns for comparison
    Q_giv = Q_giv_full[:, :A.shape[1]]
    print("\n--- Givens QR Factorization ---")
    print("Q (Givens, reduced to first n columns):")
    print(Q_giv)
    print("R (Givens):")
    print(R_giv)
    print("Reconstruction (Q_giv_full @ R_giv):")
    print(Q_giv_full @ R_giv)

    # Compare the Q matrices column by column (up to a possible sign change)
    n = A.shape[1]
    def same_up_to_sign(u, v):
        return np.allclose(u, v) or np.allclose(u, -v)

    print("\n--- Column-by-Column Comparison of Q (up to sign differences) ---")
    for i in range(n):
        col_gs = Q_gs[:, i]
        col_hh = Q_h[:, i]
        col_giv = Q_giv[:, i]
        print(f"\nColumn {i}:")
        print("Gram-Schmidt vs. Householder:", "Agree" if same_up_to_sign(col_gs, col_hh) else "Differ")
        print("Gram-Schmidt vs. Givens:", "Agree" if same_up_to_sign(col_gs, col_giv) else "Differ")


Matrix A:
[[ 1.76405235  0.40015721  0.97873798]
 [ 2.2408932   1.86755799 -0.97727788]
 [ 0.95008842 -0.15135721 -0.10321885]
 [ 0.4105985   0.14404357  1.45427351]]

--- Gram-Schmidt QR Factorization ---
Q (Gram-Schmidt):
[[ 0.581441   -0.47915974  0.25929548]
 [ 0.73861028  0.64154205 -0.15752491]
 [ 0.31315418 -0.59551882 -0.46848788]
 [ 0.13533544 -0.06470756  0.82974144]]
R (Gram-Schmidt):
[[ 3.0339318   1.58416139  0.01174224]
 [ 0.          1.08719312 -1.12857042]
 [ 0.          0.          1.66275572]]
Reconstruction (Q_gs @ R_gs):
[[ 1.76405235  0.40015721  0.97873798]
 [ 2.2408932   1.86755799 -0.97727788]
 [ 0.95008842 -0.15135721 -0.10321885]
 [ 0.4105985   0.14404357  1.45427351]]

--- Householder QR Factorization ---
Q (Householder, reduced to first n columns):
[[-0.581441    0.47915974  0.25929548]
 [-0.73861028 -0.64154205 -0.15752491]
 [-0.31315418  0.59551882 -0.46848788]
 [-0.13533544  0.06470756  0.82974144]]
R (Householder):
[[-3.03393180e+00 -1.58416139e+00 -1.17